# Build Pipeline with Components from yaml

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define and load `CommandComponent` from YAML
- Create `Pipeline` using loaded component.

**Motivations** - This notebook covers the scenario that user define components using yaml then use these components to build pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [20]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [12]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [13]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "mjn-cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: /config.json


AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'mjn-cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/b17253fa-f327-42d6-9686-f3e553e24763/resourceGroups/mjn-dpv2-cli/providers/Microsoft.MachineLearningServices/workspaces/mjn-dpv2/computes/mjn-cpu-cluster', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mjn-dpv2-ci/code/Users/manishajain/azureml-examples/mjn/pipelines/1a_pipeline_with_components_from_yaml', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7fd9852f14e0>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 10, 'idle_time_before_scale_down': 7200.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x7fd97bfc1840>, '

# 2. Define and create components into workspace
## 2.1 Load components from YAML

In [14]:
parent_dir = "."
train_model = load_component(source=parent_dir + "/train_model.yml")
score_data = load_component(source=parent_dir + "/score_data.yml")
eval_model = load_component(source=parent_dir + "/eval_model.yml")

## 2.2 Inspect loaded component

In [15]:
# Print the component as yaml
print(train_model)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
version: 0.0.1
display_name: Train Model
description: A dummy training component
type: command
inputs:
  training_data:
    type: uri_folder
  max_epochs:
    type: integer
    default: '3'
  learning_rate:
    type: number
    default: '0.01'
  learning_rate_schedule:
    type: string
    default: time-based
outputs:
  model_output:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --max_epochs
  ${{inputs.max_epochs}}  --learning_rate ${{inputs.learning_rate}}  --learning_rate_schedule
  ${{inputs.learning_rate_schedule}}  --model_output ${{outputs.model_output}}
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:5
code: ./train_src
tags: {}
is_deterministic: true



In [16]:
# Inspect more information
print(type(train_model))
help(train_model._func)

<class 'azure.ai.ml.entities._component.command_component.CommandComponent'>
Help on function [component] Train Model:

[component] Train Model(*, training_data: 'uri_folder' = None, max_epochs: 'int' = None, learning_rate: 'float' = None, learning_rate_schedule: 'str' = None)
    A dummy training component
    
    Component yaml:
    ```yaml
    $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
    name: train_model
    version: 0.0.1
    display_name: Train Model
    description: A dummy training component
    type: command
    inputs:
      training_data:
        type: uri_folder
      max_epochs:
        type: integer
        default: '3'
      learning_rate:
        type: number
        default: '0.01'
      learning_rate_schedule:
        type: string
        default: time-based
    outputs:
      model_output:
        type: uri_folder
    command: python train.py  --training_data ${{inputs.training_data}}  --max_epochs
      ${{inputs.max_epochs}

# 3. Sample pipeline job
## 3.1 Build pipeline

In [17]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    """E2E dummy train-score-eval pipeline with components defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, test_data=test_input
    )
    score_with_sample_data.outputs.score_output.mode = "upload"

    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "trained_model": train_with_sample_data.outputs.model_output,
        "scored_data": score_with_sample_data.outputs.score_output,
        "evaluation_report": eval_with_sample_data.outputs.eval_output,
    }


pipeline_job = pipeline_with_components_from_yaml(
    training_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    test_input=Input(type="uri_folder", path=parent_dir + "/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
)

# set pipeline level compute
pipeline_job.settings.default_compute = "mjn-cpu-cluster"

In [18]:
# Inspect built pipeline
print(pipeline_job)

display_name: pipeline_with_components_from_yaml
description: E2E dummy train-score-eval pipeline with components defined via yaml.
type: pipeline
inputs:
  training_input:
    type: uri_folder
    path: azureml:./data/
  test_input:
    type: uri_folder
    path: azureml:./data/
  training_max_epochs: 20
  training_learning_rate: 1.8
  learning_rate_schedule: time-based
outputs:
  trained_model: null
  scored_data: null
  evaluation_report: null
jobs:
  train_with_sample_data:
    type: command
    inputs:
      training_data:
        path: ${{parent.inputs.training_input}}
      max_epochs:
        path: ${{parent.inputs.training_max_epochs}}
      learning_rate:
        path: ${{parent.inputs.training_learning_rate}}
      learning_rate_schedule:
        path: ${{parent.inputs.learning_rate_schedule}}
    outputs:
      model_output: ${{parent.outputs.trained_model}}
    code: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mjn-dpv2-ci/code/Users/manishajain/azureml-examples/mjn/pip

## 3.2 Submit pipeline job

In [19]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline_job, experiment_name="mjn-pipeline_samples")
pipeline_job

Experiment,Name,Type,Status,Details Page
mjn-pipeline_samples,quirky_tail_s24xv710st,pipeline,Preparing,Link to Azure Machine Learning studio


In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../README.md)